In [1]:
import pandas as pd
from collections import Counter
import numpy as np
from copy import deepcopy
import random
from functools import reduce
import pickle

In [2]:
train_replace_dict = {'atis_flight#atis_airfare': 'atis_airfare', 
                                     'atis_ground_service#atis_ground_fare': 'atis_ground_fare',
                                     'atis_aircraft#atis_flight#atis_flight_no': 'atis_flight_no',
                                     'atis_airfare#atis_flight_time':'atis_flight_time',
                                     'atis_airline#atis_flight_no':'atis_flight_no',
                                     'atis_flight_no#atis_airline' : 'atis_flight_no',
                                     'atis_airfare#atis_flight': 'atis_airfare',
                                     'atis_flight#atis_airline' : 'atis_airline'}

# intent to idx mapping
intent_list = (pd.read_csv('../data/multiATIS/intent_list.csv',sep=',',header=None,names=['INTENT']).INTENT.values.tolist())
intent2idx = { intent:idx for idx,intent in enumerate(intent_list)}

# slot idx mapping
final_slots = pd.read_csv('../data/multiATIS/slots_list.csv',sep=',',header=None,names=['SLOTS']).SLOTS.values.tolist()
slots2idx  = {slots:idx for idx,slots in enumerate(final_slots)}
idx2slots = {v: k for k, v in slots2idx.items()}

In [3]:
def mapping2idx(df,fn):
    
    #del df['id']

    # adding intent encoding
    encoded_intent = []
    for intent in list(df.INTENT):
        try:
            encoded_intent.append(intent2idx[intent])
        except:
            print(intent)
            encoded_intent.append(intent2idx['OOI'])
            
        
    df['INTENT_ID'] = encoded_intent 

    # adding mapped slot column
    slots_ID = []

    for annotations in list(df.SLOTS):
        encoded_slot = ""
        #print((annotations.split()))
        for tokens in annotations.split():
            try: 
                encoded_slot += str(slots2idx[tokens]) + " "
            except:
                print('token',tokens,fn)
                encoded_slot += str(slots2idx['O'])
        slots_ID.append(" ".join(encoded_slot.split()))

    df['SLOTS_ID'] = slots_ID
    
    return df

In [4]:
def get_carrier_phrase_length(annotations):
    CP_idx = []
    for idx,token in enumerate(annotations.split(' ')):
        if token == 'O':
            CP_idx.append(idx)
    
    return CP_idx
    

In [5]:
def carrier_aug(data,tau):
    
    orig_data = deepcopy(data)
    
    augINTENT, augSLOTS, augTEXT,augID = [],[],[],[]
    
    cnt = 0
    for sample in data.values.tolist():
        
        
        CP_idx = get_carrier_phrase_length(sample[2])
        
        CP_length = len(CP_idx)
        
        if CP_length <= 1:
            del_count = CP_length
        else:
            del_count = 1 if CP_length <=3 else int(tau*CP_length)
        
        del_index = random.sample(CP_idx,del_count)

        augINTENT.append(sample[3])
        TEXT = ' '.join([i for j, i in enumerate(sample[1].split(' ')) if j not in del_index])
        SLOTS = ' '.join([i for j, i in enumerate(sample[2].split(' ')) if j not in del_index])
        augTEXT.append(TEXT)
        augSLOTS.append(SLOTS)
        augID.append(cnt)
        cnt+=1
    
               
    augPD = pd.DataFrame([augID,augTEXT,augSLOTS,augINTENT],index=['ID','TEXT','SLOTS','INTENT']).T
    
    return augPD

In [6]:
def aug_data(data_path,out_path,fn,tau):
    
    # loading dataset
    data = pd.read_csv(data_path,sep='\t',header=0,names=['ID','TEXT','SLOTS','INTENT'])
    
    # lowerCasing the TEXT column
    data['TEXT'] = data['TEXT'].str.lower()
    
    # handling multi-label instances 
    data['INTENT'] = data['INTENT'].replace(train_replace_dict)
    
    aug_data = carrier_aug(data,tau=tau)
    
    #print(aug_data.INTENT)
    
    aug_data = mapping2idx(aug_data,fn)
    
    #print(len(data),len(aug_data))
    
    aug_data = aug_data.sample(frac=1).reset_index(drop=True)
    aug_data.to_csv(out_path,sep='\t',index=False)
    

In [ ]:
# processing EN training data
process_data('../data/multiATIS/raw/train_dev_test/test_EN.tsv','../data/multiATIS/splits/test/OOC_test/test_EN.tsv','EN')
# processing ES training data
process_data('../data/multiATIS/raw/train_dev_test/test_ES.tsv','../data/multiATIS/splits/test/OOC_test/test_ES.tsv','ES')
# processing DE training data
process_data('../data/multiATIS/raw/train_dev_test/test_DE.tsv','../data/multiATIS/splits/test/OOC_test/test_DE.tsv','DE')
# processing FR training data
process_data('../data/multiATIS/raw/train_dev_test/test_FR.tsv','../data/multiATIS/splits/test/OOC_test/test_FR.tsv','FR')

In [7]:
aug_data('../data/multiATIS/raw/train_dev_test/test_DE.tsv','../data/multiATIS/split/test/OOC_test/0_25n/test_DE.tsv','DE',tau=0.25)
aug_data('../data/multiATIS/raw/train_dev_test/test_DE.tsv','../data/multiATIS/split/test/OOC_test/0_50n/test_DE.tsv','DE',tau=0.50)
aug_data('../data/multiATIS/raw/train_dev_test/test_DE.tsv','../data/multiATIS/split/test/OOC_test/0_10n/test_DE.tsv','DE',tau=0.10)
aug_data('../data/multiATIS/raw/train_dev_test/test_DE.tsv','../data/multiATIS/split/test/OOC_test/0_75n/test_DE.tsv','DE',tau=0.75)

aug_data('../data/multiATIS/raw/train_dev_test/test_EN.tsv','../data/multiATIS/split/test/OOC_test/0_25n/test_EN.tsv','DE',tau=0.25)
aug_data('../data/multiATIS/raw/train_dev_test/test_EN.tsv','../data/multiATIS/split/test/OOC_test/0_50n/test_EN.tsv','DE',tau=0.50)
aug_data('../data/multiATIS/raw/train_dev_test/test_EN.tsv','../data/multiATIS/split/test/OOC_test/0_10n/test_EN.tsv','DE',tau=0.10)
aug_data('../data/multiATIS/raw/train_dev_test/test_EN.tsv','../data/multiATIS/split/test/OOC_test/0_75n/test_EN.tsv','DE',tau=0.75)

aug_data('../data/multiATIS/raw/train_dev_test/test_ES.tsv','../data/multiATIS/split/test/OOC_test/0_25n/test_ES.tsv','DE',tau=0.25)
aug_data('../data/multiATIS/raw/train_dev_test/test_ES.tsv','../data/multiATIS/split/test/OOC_test/0_50n/test_ES.tsv','DE',tau=0.50)
aug_data('../data/multiATIS/raw/train_dev_test/test_ES.tsv','../data/multiATIS/split/test/OOC_test/0_10n/test_ES.tsv','DE',tau=0.10)
aug_data('../data/multiATIS/raw/train_dev_test/test_ES.tsv','../data/multiATIS/split/test/OOC_test/0_75n/test_ES.tsv','DE',tau=0.75)

aug_data('../data/multiATIS/raw/train_dev_test/test_FR.tsv','../data/multiATIS/split/test/OOC_test/0_25n/test_FR.tsv','DE',tau=0.25)
aug_data('../data/multiATIS/raw/train_dev_test/test_FR.tsv','../data/multiATIS/split/test/OOC_test/0_50n/test_FR.tsv','DE',tau=0.50)
aug_data('../data/multiATIS/raw/train_dev_test/test_FR.tsv','../data/multiATIS/split/test/OOC_test/0_10n/test_FR.tsv','DE',tau=0.10)
aug_data('../data/multiATIS/raw/train_dev_test/test_FR.tsv','../data/multiATIS/split/test/OOC_test/0_75n/test_FR.tsv','DE',tau=0.75)

atis_day_name
atis_day_name
token B-compartment DE
token B-stoploc.airport_code DE
token B-booking_class DE
token B-flight DE
atis_day_name
atis_day_name
token B-compartment DE
token B-stoploc.airport_code DE
token B-booking_class DE
token B-flight DE
atis_day_name
atis_day_name
token B-compartment DE
token B-stoploc.airport_code DE
token B-booking_class DE
token B-flight DE
atis_day_name
atis_day_name
token B-compartment DE
token B-stoploc.airport_code DE
token B-booking_class DE
token B-flight DE


# Wake word model Time limit exceed noise

In [ ]:
def WWTLE_noise(data):
    
    orig_data = deepcopy(data)
    
    augINTENT, augSLOTS, augTEXT,augID = [],[],[],[]
    
    cnt = 0
    for sample in data.values.tolist():
        
        CP_length = len(sample.split())
        
        if CP_length <= 7:
            
        else:
            trim_index = random.sample(CP_idx,del_count)

        augINTENT.append(sample[3])
        TEXT = ' '.join([i for j, i in enumerate(sample[1].split(' ')) if j not in del_index])
        SLOTS = ' '.join([i for j, i in enumerate(sample[2].split(' ')) if j not in del_index])
        augTEXT.append(TEXT)
        augSLOTS.append(SLOTS)
        augID.append(cnt)
        cnt+=1
    
               
    augPD = pd.DataFrame([augID,augTEXT,augSLOTS,augINTENT],index=['ID','TEXT','SLOTS','INTENT']).T
    
    return augPD

In [ ]:
def WWTLE_aug_data(data_path,out_path,fn,tau):
    
    # loading dataset
    data = pd.read_csv(data_path,sep='\t',header=0,names=['ID','TEXT','SLOTS','INTENT'])
    
    # lowerCasing the TEXT column
    data['TEXT'] = data['TEXT'].str.lower()
    
    # handling multi-label instances 
    data['INTENT'] = data['INTENT'].replace(train_replace_dict)
    
    aug_data = carrier_aug(data,tau=tau)
    
    #print(aug_data.INTENT)
    
    aug_data = mapping2idx(aug_data,fn)
    
    #print(len(data),len(aug_data))
    
    aug_data = aug_data.sample(frac=1).reset_index(drop=True)
    aug_data.to_csv(out_path,sep='\t',index=False)
    